In [74]:
using LowLevelFEM

In [75]:
gmsh.initialize()

In [76]:
function cube_mesh(; lx=1.0, ly=1.0, lz=1.0, n=10, dx=lx / n, dy=ly / n, dz=lz / n)

    gmsh.option.setNumber("General.Verbosity", 0)

    # --------------------------------------------------
    # Geometry
    # --------------------------------------------------
    # Box: origin (0,0,0), size l x l x l
    box = gmsh.model.occ.addBox(0.0, 0.0, 0.0, lx, ly, lz)

    gmsh.model.occ.synchronize()
    #return

    # --------------------------------------------------
    # Physical groups
    # --------------------------------------------------
    gmsh.model.addPhysicalGroup(2, [1], -1, "left")
    gmsh.model.addPhysicalGroup(2, [2], -1, "right")
    gmsh.model.addPhysicalGroup(2, [6], -1, "front")
    gmsh.model.addPhysicalGroup(2, [5], -1, "rear")
    gmsh.model.addPhysicalGroup(2, [3], -1, "bottom")
    gmsh.model.addPhysicalGroup(2, [4], -1, "top")

    gmsh.model.addPhysicalGroup(3, [1], -1, "volume")

    # --------------------------------------------------
    # Mesh settings (structured hex mesh)
    # --------------------------------------------------
    gmsh.option.setNumber("Mesh.Algorithm3D", 1)      # Delaunay for recombination
    gmsh.option.setNumber("Mesh.RecombineAll", 1)
    gmsh.option.setNumber("Mesh.RecombinationAlgorithm", 1)

    gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)

    gmsh.model.mesh.setTransfiniteAutomatic()

    gmsh.option.setNumber("Mesh.ElementOrder", 1)

    # Characteristic length control
    lc = min(dx, dy, dz)
    gmsh.option.setNumber("Mesh.CharacteristicLengthMin", lc)
    gmsh.option.setNumber("Mesh.CharacteristicLengthMax", lc)

    # --------------------------------------------------
    # Generate mesh
    # --------------------------------------------------
    gmsh.model.mesh.generate(3)

    # --------------------------------------------------
    # Save
    # --------------------------------------------------
    #gmsh.write("cube.msh")

    #gmsh.finalize()

    return nothing
end


cube_mesh (generic function with 1 method)

In [77]:
cube_mesh()

In [78]:
mat = Material("volume")
prob = Problem([mat])

Problem("", :Solid, 3, 3, Material[Material("volume", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 512, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [79]:
C = [1 2 2 0 0 0;
    2 0.2 2 0 0 0;
    2 2 1 0 0 0;
    0 0 0 1 0 0;
    0 0 0 0 1 0;
    0 0 0 0 0 1]

6×6 Matrix{Float64}:
 1.0  2.0  2.0  0.0  0.0  0.0
 2.0  0.2  2.0  0.0  0.0  0.0
 2.0  2.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0

In [80]:
F = TensorField(prob, "volume", [1 0 0; 0 1 0; 0 0 1])

elementwise TensorField
[[1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;]  …  [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;]]

In [81]:
K = materialTangentMatrix(prob, F=F, C=C)

sparse([1, 2, 3, 40, 41, 42, 43, 44, 45, 187  …  1515, 1516, 1517, 1518, 1531, 1532, 1533, 1534, 1535, 1536], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1536, 1536, 1536, 1536, 1536, 1536, 1536, 1536, 1536, 1536], [0.047619047619047575, 0.0357142857142857, -0.035714285714285685, 4.098284211995207e-17, 0.017857142857142884, 0.011904761904761888, -1.3145951341386741e-17, -0.011904761904761897, -0.017857142857142853, -3.0357660829594124e-18  …  -0.023809523809523878, -3.469446951953614e-18, 3.400058012914542e-16, -3.838346741141807e-16, -7.112366251504909e-17, 4.163336342344337e-17, 9.278060091044704e-17, -6.938893903907228e-17, 3.400058012914542e-16, 0.3809523809523809], 1536, 1536)

In [82]:
K.A

1536×1536 SparseArrays.SparseMatrixCSC{Float64, Int64} with 95158 stored entries:
⎡⣿⣿⣛⠒⠚⣟⣻⣦⣬⡗⠀⠘⠇⠀⢚⡁⠀⠚⠀⢘⢷⣤⣈⠓⠀⡳⠀⠀⡰⠀⢀⡄⠀⢀⠄⠀⠨⣧⣄⡑⎤
⎢⢻⠘⠻⣦⣚⢞⡙⠈⠉⢓⣶⣮⠁⠀⠾⠁⠀⣘⠛⠓⡠⠤⣬⡟⠛⢃⠂⠀⢀⠆⠈⡱⠀⠈⡰⠀⢈⠤⢤⣥⎥
⎢⣾⢤⣺⢜⣿⣿⣣⠀⠠⣟⠈⠹⠶⠶⢧⣄⣀⡻⠉⢹⡧⠀⢀⠿⠉⣿⠀⠀⠀⠀⠀⠀⠀⠀⠁⠀⠨⠄⠀⡈⎥
⎢⠻⣾⡓⠈⠉⠚⢿⣷⣄⠁⠀⠘⡇⠀⢰⡉⠉⠙⠓⠒⠀⠀⠈⠿⠶⠾⣷⣶⣶⣶⣤⣤⣄⣀⣀⣀⠀⠀⠀⠉⎥
⎢⢦⠿⢧⢀⣤⢦⠄⠙⠿⣧⣄⠀⠃⠀⣜⠇⠀⡠⠤⣤⡒⠶⠦⣤⣤⣄⣤⣤⣄⣉⣉⡉⠛⠛⠛⠿⠿⠻⠷⠆⎥
⎢⣀⠀⡸⣿⣆⡀⣀⠀⠀⠙⠿⣧⣀⠀⢻⠀⠀⣧⠀⠀⢀⣀⣀⡀⠈⢉⡉⠉⠛⠛⠻⠿⠿⠿⢷⣶⣶⣤⣤⣤⎥
⎢⠉⠁⠁⠀⢸⡇⠉⠉⠉⠀⠀⠘⢿⣷⣄⠀⠀⢸⠀⠀⣷⠀⠀⣇⠀⠸⡇⠀⢸⡄⠀⢻⠀⠀⣧⠀⠘⡇⠀⠀⎥
⎢⠞⠰⠞⠃⠉⢷⡔⠲⠶⠝⠛⠒⠀⠙⢻⣶⣄⠈⠀⢰⡜⠀⢠⠉⠀⣶⠃⠀⡞⠃⢰⡞⠀⢰⡙⠀⢰⠃⠀⣄⎥
⎢⣠⠀⣀⢠⣤⡸⣇⠀⠀⡠⠤⣤⣀⣀⡀⠙⢿⣷⡀⠀⠇⠀⢸⣄⡀⠸⠀⠀⠻⠀⠀⠇⠀⠘⠇⠀⢸⠀⠀⢻⎥
⎢⣀⢀⢿⠀⣇⣀⢹⠀⠀⣧⠀⠀⠀⠀⢀⣀⠀⠈⢿⣷⡄⠀⠀⠙⣿⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠙⣷⠀⡎⠉⠋⠀⠀⢸⡌⠀⢰⠙⠛⠒⠉⠉⠁⠀⠉⢿⣷⣄⠀⠀⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⢿⣦⡀⎥
⎢⢦⠘⣦⠿⣤⡔⣦⡄⠈⣧⠀⠸⠤⢤⡄⠒⠒⢶⣄⠀⠀⠙⠿⣧⡄⢠⣤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠿⎥
⎢⢤⡠⠿⢀⣧⣤⣸⡇⠀⢿⡆⢀⣀⡀⢠⣤⣀⡈⠻⣿⡄⠀⠀⣉⢿⣷⣙⢿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠈⠀⠀⠀⢹⣿⠀⣿⡇⠈⠉⠉⠉⠀⠀⠀⠀⠀⠀⠀⠀⠻⣷⣜⢿⣷⣜⢿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠐⠊⠠⠔⠀⠀⢸⣿⡄⢹⣿⠀⠒⠶⠾⠉⠛⠂⠀⠀⠀⠀⠀⠀⠈⠻⣶⣝⢻⣶⡝⢷⣦⡀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠴⢆⡠⠀⠀⠀⣿⡇⠸⣿⡆⣤⣀⣰⠶⠤⠄⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⢷⣍⠻⣦⣝⠿⣦⡀⠀⠀⠀⠀⎥
⎢⠀⢀⡀⠀⠀⠀⠀⢹⣿⠀⣿⡇⠀⠀⢀⣀⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣷⡝⢿⣷⡝⢿⣦⠀⠀⠀⎥
⎢⠀⠁⠐⠊⠁⠀⠀⢸⣿⡄⢹⣷⠉⠛⠓⠈⠉⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣷⣍⢿⣷⣝⢿⣦⡀⎥
⎢⠦⣦⠂⡔⠂⠆⠀⠀⣿⡃⠘⣿⠶⠤⠴⠒⠒⠒⠀⠀⣶⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⣷⣝⢻⣶⣝⠛⎥
⎣⢄⠹⠄⣷⡀⠠⡄⠀⠹⠇⠀⣿⠀⠀⠀⢤⣤⣀⠀⠀⠈⠻⣷⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣷⠙⢿⣷⎦

In [83]:
suppX = BoundaryCondition("left", ux=0)
suppY = BoundaryCondition("bottom", uy=0)
suppZ = BoundaryCondition("rear", uz=0)

load = BoundaryCondition("right", fx=-1)

BoundaryCondition("right", nothing, nothing, nothing, nothing, nothing, nothing, -1, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [84]:
f = loadVector(prob, [load])

nodal VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [85]:
u = solveDisplacement(K, f, support=[suppX, suppY, suppZ])

nodal VectorField
[0.0; 0.0; … ; -0.23166023165932234; -0.41698841698895955;;]

In [86]:
showDoFResults(u, name="u")

0

In [87]:
openPostProcessor()

In [88]:
gmsh.finalize()